# Question Classifier Node Test Notebook

This notebook tests the `classifier.py` module functionality with various question types.

In [1]:
from dotenv import load_dotenv, find_dotenv

# .env 파일 로드
dotenv_path = find_dotenv()
load_dotenv(dotenv_path, override=True)

True

In [2]:
# Import required modules
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from app.agents.nodes.classifier import classify_question
from app.agents.prompts.classification import VALID_QUESTION_TYPES, DEFAULT_QUESTION_TYPE
from app.agents.state import ChatState

/Users/kimjunghyeon/Desktop/workspace/ai-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test Setup

Create mock chat states with different question types for testing

In [3]:
# Test Case 1: FACT questions
def create_fact_question_states():
    return [
        {
            "messages": [HumanMessage(content="파이썬이란 무엇인가요?", id="msg1")],
            "expected": "FACT",
            "description": "Basic fact question"
        },
        {
            "messages": [
                AIMessage(content="안녕하세요!", id="msg1"),
                HumanMessage(content="FastAPI의 주요 특징은 무엇인가요?", id="msg2")
            ],
            "expected": "FACT",
            "description": "Technical fact question"
        },
        {
            "messages": [
                HumanMessage(content="Docker 컨테이너란?", id="msg1")
            ],
            "expected": "FACT",
            "description": "Definition question"
        }
    ]

# Test Case 2: SUMMARY questions
def create_summary_question_states():
    return [
        {
            "messages": [HumanMessage(content="파이썬 프로그래밍 언어에 대해 요약해주세요", id="msg1")],
            "expected": "SUMMARY",
            "description": "Direct summary request"
        },
        {
            "messages": [HumanMessage(content="이 문서의 내용을 간단히 정리해주세요", id="msg1")],
            "expected": "SUMMARY",
            "description": "Document summary request"
        },
        {
            "messages": [
                AIMessage(content="네, 도와드리겠습니다.", id="msg1"),
                HumanMessage(content="지금까지 논의한 내용을 요약해주세요", id="msg2")
            ],
            "expected": "SUMMARY",
            "description": "Conversation summary request"
        }
    ]

# Test Case 3: COMPARE questions
def create_compare_question_states():
    return [
        {
            "messages": [HumanMessage(content="Python과 Java의 차이점은 무엇인가요?", id="msg1")],
            "expected": "COMPARE",
            "description": "Language comparison"
        },
        {
            "messages": [HumanMessage(content="Django와 FastAPI 중 어떤 것이 더 좋은가요?", id="msg1")],
            "expected": "COMPARE",
            "description": "Framework comparison with preference"
        },
        {
            "messages": [
                HumanMessage(content="안녕하세요", id="msg1"),
                AIMessage(content="안녕하세요! 도와드릴게요.", id="msg2"),
                HumanMessage(content="MySQL과 PostgreSQL을 비교해주세요", id="msg3")
            ],
            "expected": "COMPARE",
            "description": "Database comparison in conversation"
        }
    ]

# Test Case 4: EVIDENCE questions
def create_evidence_question_states():
    return [
        {
            "messages": [HumanMessage(content="왜 Python이 인기 있는 언어인지 근거를 제시해주세요", id="msg1")],
            "expected": "EVIDENCE",
            "description": "Evidence for popularity"
        },
        {
            "messages": [HumanMessage(content="이 주장을 뒷받침하는 증거가 있나요?", id="msg1")],
            "expected": "EVIDENCE",
            "description": "Evidence request"
        },
        {
            "messages": [HumanMessage(content="이런 결론을 내린 근거는 무엇인가요?", id="msg1")],
            "expected": "EVIDENCE",
            "description": "Reasoning evidence request"
        }
    ]

# Test Case 5: Edge cases
def create_edge_case_states():
    return [
        {
            "messages": [],
            "expected": DEFAULT_QUESTION_TYPE,
            "description": "Empty messages"
        },
        {
            "messages": [AIMessage(content="안녕하세요!", id="msg1")],
            "expected": DEFAULT_QUESTION_TYPE,
            "description": "No human message"
        },
        {
            "messages": [HumanMessage(content="", id="msg1")],
            "expected": DEFAULT_QUESTION_TYPE,
            "description": "Empty human message"
        },
        {
            "messages": [HumanMessage(content="안녕하세요", id="msg1")],
            "expected": DEFAULT_QUESTION_TYPE,
            "description": "Ambiguous greeting"
        }
    ]

print("Test data created successfully!")
print(f"Valid question types: {VALID_QUESTION_TYPES}")
print(f"Default question type: {DEFAULT_QUESTION_TYPE}")

Test data created successfully!
Valid question types: {'FACT', 'COMPARE', 'EVIDENCE', 'SUMMARY'}
Default question type: FACT


## Test Helper Functions

In [4]:
async def run_classification_test(test_cases, category_name):
    """
    Run classification tests for a specific category
    """
    print(f"\n=== {category_name} Test Cases ===")
    
    results = []
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {test_case['description']}")
        
        # Show input
        messages = test_case['messages']
        if messages:
            last_human = next(
                (msg.content for msg in reversed(messages) if isinstance(msg, HumanMessage)),
                "No human message"
            )
            print(f"  Input: '{last_human}'")
        else:
            print(f"  Input: Empty messages")
        
        # Run classification
        try:
            result = await classify_question(test_case)
            predicted = result.get('question_type')
            model_used = result.get('model_used')
            
            expected = test_case['expected']
            is_correct = predicted == expected
            
            print(f"  Expected: {expected}")
            print(f"  Predicted: {predicted}")
            print(f"  Model: {model_used}")
            print(f"  ✅ Correct" if is_correct else f"  ❌ Incorrect")
            
            results.append({
                'test_case': test_case['description'],
                'expected': expected,
                'predicted': predicted,
                'correct': is_correct
            })
            
        except Exception as e:
            print(f"  ❌ Error: {e}")
            results.append({
                'test_case': test_case['description'],
                'expected': test_case['expected'],
                'predicted': 'ERROR',
                'correct': False
            })
    
    # Summary
    correct_count = sum(1 for r in results if r['correct'])
    total_count = len(results)
    accuracy = correct_count / total_count if total_count > 0 else 0
    
    print(f"\n{category_name} Results: {correct_count}/{total_count} ({accuracy:.1%}) correct")
    
    return results

## Test Case 1: FACT Questions

Test classification of factual information questions

In [5]:
fact_results = await run_classification_test(create_fact_question_states(), "FACT")


=== FACT Test Cases ===

Test 1: Basic fact question
  Input: '파이썬이란 무엇인가요?'
  Expected: FACT
  Predicted: FACT
  Model: gpt-4o-mini
  ✅ Correct

Test 2: Technical fact question
  Input: 'FastAPI의 주요 특징은 무엇인가요?'
  Expected: FACT
  Predicted: FACT
  Model: gpt-4o-mini
  ✅ Correct

Test 3: Definition question
  Input: 'Docker 컨테이너란?'
  Expected: FACT
  Predicted: FACT
  Model: gpt-4o-mini
  ✅ Correct

FACT Results: 3/3 (100.0%) correct


## Test Case 2: SUMMARY Questions

Test classification of summary request questions

In [6]:
summary_results = await run_classification_test(create_summary_question_states(), "SUMMARY")


=== SUMMARY Test Cases ===

Test 1: Direct summary request
  Input: '파이썬 프로그래밍 언어에 대해 요약해주세요'
  Expected: SUMMARY
  Predicted: SUMMARY
  Model: gpt-4o-mini
  ✅ Correct

Test 2: Document summary request
  Input: '이 문서의 내용을 간단히 정리해주세요'
  Expected: SUMMARY
  Predicted: SUMMARY
  Model: gpt-4o-mini
  ✅ Correct

Test 3: Conversation summary request
  Input: '지금까지 논의한 내용을 요약해주세요'
  Expected: SUMMARY
  Predicted: SUMMARY
  Model: gpt-4o-mini
  ✅ Correct

SUMMARY Results: 3/3 (100.0%) correct


## Test Case 3: COMPARE Questions

Test classification of comparison questions

In [7]:
compare_results = await run_classification_test(create_compare_question_states(), "COMPARE")


=== COMPARE Test Cases ===

Test 1: Language comparison
  Input: 'Python과 Java의 차이점은 무엇인가요?'
  Expected: COMPARE
  Predicted: COMPARE
  Model: gpt-4o-mini
  ✅ Correct

Test 2: Framework comparison with preference
  Input: 'Django와 FastAPI 중 어떤 것이 더 좋은가요?'
  Expected: COMPARE
  Predicted: COMPARE
  Model: gpt-4o-mini
  ✅ Correct

Test 3: Database comparison in conversation
  Input: 'MySQL과 PostgreSQL을 비교해주세요'
  Expected: COMPARE
  Predicted: COMPARE
  Model: gpt-4o-mini
  ✅ Correct

COMPARE Results: 3/3 (100.0%) correct


## Test Case 4: EVIDENCE Questions

Test classification of evidence/reasoning questions

In [8]:
evidence_results = await run_classification_test(create_evidence_question_states(), "EVIDENCE")


=== EVIDENCE Test Cases ===

Test 1: Evidence for popularity
  Input: '왜 Python이 인기 있는 언어인지 근거를 제시해주세요'
  Expected: EVIDENCE
  Predicted: EVIDENCE
  Model: gpt-4o-mini
  ✅ Correct

Test 2: Evidence request
  Input: '이 주장을 뒷받침하는 증거가 있나요?'
  Expected: EVIDENCE
  Predicted: EVIDENCE
  Model: gpt-4o-mini
  ✅ Correct

Test 3: Reasoning evidence request
  Input: '이런 결론을 내린 근거는 무엇인가요?'
  Expected: EVIDENCE
  Predicted: EVIDENCE
  Model: gpt-4o-mini
  ✅ Correct

EVIDENCE Results: 3/3 (100.0%) correct


## Test Case 5: Edge Cases

Test classification of edge cases and ambiguous inputs

In [9]:
edge_results = await run_classification_test(create_edge_case_states(), "EDGE CASES")


=== EDGE CASES Test Cases ===

Test 1: Empty messages
  Input: Empty messages
  Expected: FACT
  Predicted: FACT
  Model: gpt-4o-mini
  ✅ Correct

Test 2: No human message
  Input: 'No human message'
  Expected: FACT
  Predicted: FACT
  Model: gpt-4o-mini
  ✅ Correct

Test 3: Empty human message
  Input: ''
  Expected: FACT
  Predicted: FACT
  Model: gpt-4o-mini
  ✅ Correct

Test 4: Ambiguous greeting
  Input: '안녕하세요'
  Expected: FACT
  Predicted: FACT
  Model: gpt-4o-mini
  ✅ Correct

EDGE CASES Results: 4/4 (100.0%) correct


## Overall Test Results Summary

In [10]:
# Combine all results
all_results = fact_results + summary_results + compare_results + evidence_results + edge_results

print("\n" + "="*50)
print("OVERALL TEST RESULTS SUMMARY")
print("="*50)

# Overall accuracy
total_correct = sum(1 for r in all_results if r['correct'])
total_tests = len(all_results)
overall_accuracy = total_correct / total_tests if total_tests > 0 else 0

print(f"\nTotal Tests: {total_tests}")
print(f"Correct: {total_correct}")
print(f"Incorrect: {total_tests - total_correct}")
print(f"Overall Accuracy: {overall_accuracy:.1%}")

# Accuracy by category
print(f"\nAccuracy by Category:")
categories = [
    ("FACT", fact_results),
    ("SUMMARY", summary_results),
    ("COMPARE", compare_results),
    ("EVIDENCE", evidence_results),
    ("EDGE CASES", edge_results)
]

for category_name, results in categories:
    if results:
        correct = sum(1 for r in results if r['correct'])
        total = len(results)
        accuracy = correct / total if total > 0 else 0
        print(f"  {category_name}: {correct}/{total} ({accuracy:.1%})")

# Show failed cases
failed_cases = [r for r in all_results if not r['correct']]
if failed_cases:
    print(f"\nFailed Test Cases:")
    for case in failed_cases:
        print(f"  - {case['test_case']}: Expected {case['expected']}, Got {case['predicted']}")
else:
    print(f"\n🎉 All test cases passed!")

print("\n" + "="*50)


OVERALL TEST RESULTS SUMMARY

Total Tests: 16
Correct: 16
Incorrect: 0
Overall Accuracy: 100.0%

Accuracy by Category:
  FACT: 3/3 (100.0%)
  SUMMARY: 3/3 (100.0%)
  COMPARE: 3/3 (100.0%)
  EVIDENCE: 3/3 (100.0%)
  EDGE CASES: 4/4 (100.0%)

🎉 All test cases passed!



## Manual Test: Interactive Classification

Test custom questions interactively

In [11]:
async def test_custom_question(question_text):
    """
    Test classification of a custom question
    """
    state = {
        "messages": [HumanMessage(content=question_text, id="custom")]
    }
    
    print(f"Question: '{question_text}'")
    
    try:
        result = await classify_question(state)
        print(f"Classification: {result.get('question_type')}")
        print(f"Model: {result.get('model_used')}")
    except Exception as e:
        print(f"Error: {e}")

# Test some custom questions
custom_questions = [
    "머신러닝이란 무엇인가요?",
    "이 데이터를 요약해주세요",
    "A와 B 중 어떤 것이 더 나은가요?",
    "이 결과가 정확한 이유는 무엇인가요?"
]

print("=== Custom Question Tests ===")
for i, question in enumerate(custom_questions, 1):
    print(f"\nCustom Test {i}:")
    await test_custom_question(question)

=== Custom Question Tests ===

Custom Test 1:
Question: '머신러닝이란 무엇인가요?'
Classification: FACT
Model: gpt-4o-mini

Custom Test 2:
Question: '이 데이터를 요약해주세요'
Classification: SUMMARY
Model: gpt-4o-mini

Custom Test 3:
Question: 'A와 B 중 어떤 것이 더 나은가요?'
Classification: COMPARE
Model: gpt-4o-mini

Custom Test 4:
Question: '이 결과가 정확한 이유는 무엇인가요?'
Classification: EVIDENCE
Model: gpt-4o-mini
